In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from tqdm import tqdm
from glob import glob
import gc
import cv2
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import math
import zipfile
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime

import keras
import pydot
import pydotplus
from pydotplus import graphviz
from keras.utils.vis_utils import plot_model
from keras.utils.vis_utils import model_to_dot
keras.utils.vis_utils.pydot = pydot
from tensorflow.keras.utils import plot_model
import importlib

# import scipy.misc as misc
# from scipy.misc import imresize

In [15]:
BATCH_SIZE = 32
img_size = 256
#weights = None
weights = 'imagenet'
learning_rate = 1e-5
EPOCHS = 2
dropout_rate = 0.5

## 1. 데이터

In [16]:
train_csv = pd.read_csv('./data/train.csv')
test_csv = pd.read_csv('./data/test.csv')
train_csv.head(), test_csv.head()

(   img_id              input_img              label_img
 0   10000  train_input_10000.png  train_label_10000.png
 1   10001  train_input_10001.png  train_label_10001.png
 2   10002  train_input_10002.png  train_label_10002.png
 3   10003  train_input_10003.png  train_label_10003.png
 4   10004  train_input_10004.png  train_label_10004.png,
    img_id             input_img submission_name
 0   20000  test_input_20000.png  test_20000.png
 1   20001  test_input_20001.png  test_20001.png
 2   20002  test_input_20002.png  test_20002.png
 3   20003  test_input_20003.png  test_20003.png
 4   20004  test_input_20004.png  test_20004.png)

In [17]:
train_all_input_files = './data/train_input_img/'+train_csv['input_img']
train_all_label_files = './data/train_label_img/'+train_csv['label_img']

In [18]:
from sklearn.model_selection import train_test_split

# Assuming train_all_input_files and train_all_label_files are NumPy arrays
train_input_files, val_input_files, train_label_files, val_label_files = train_test_split(
    train_all_input_files.to_numpy(),
    train_all_label_files.to_numpy(),
    test_size=0.1,  # Adjust the test_size parameter to control the validation set size
    random_state=42  # Set a random seed for reproducibility
)

# Now you have randomly split data
print("Train set size:", len(train_input_files), len(train_label_files))
print("Validation set size:", len(val_input_files), len(val_label_files))

Train set size: 559 559
Validation set size: 63 63


### 전처리

1. 이미지를 슬라이딩하며 256x256으로 잘라 데이터셋 구성

In [19]:
def cut_img(img_path_list, save_path, stride):
    os.makedirs(f'{save_path}{img_size}', exist_ok=True)
    num = 0
    for path in tqdm(img_path_list):
        img = cv2.imread(path)
        for top in range(0, img.shape[0], stride):
            for left in range(0, img.shape[1], stride):
                piece = np.zeros([img_size, img_size, 3], np.uint8)
                temp = img[top:top+img_size, left:left+img_size, :]
                piece[:temp.shape[0], :temp.shape[1], :] = temp
                np.save(f'{save_path}{img_size}/{num}.npy', piece)
                num+=1

In [20]:
#cut_img(train_input_files, './data/train_input_img_', 128)
#cut_img(train_label_files, './data/train_label_img_', 128)
#cut_img(val_input_files, './data/val_input_img_', 128)
#cut_img(val_label_files, './data/val_label_img_', 128)

In [21]:
train_inp_files = glob(f'./data/train_input_img_{img_size}/*.npy')
train_targ_files = glob(f'./data/train_label_img_{img_size}/*.npy')

val_inp_files = glob(f'./data/val_input_img_{img_size}/*.npy')
val_targ_files = glob(f'./data/val_label_img_{img_size}/*.npy')

In [22]:
train_inp_files, train_targ_files = shuffle(train_inp_files, train_targ_files, random_state=42)
len(train_inp_files), len(train_targ_files),len(val_inp_files), len(val_targ_files)

(167050, 167050, 19890, 19890)

2. 정규화 + 증강

In [23]:
#train 0 ~ 255를 0~1로 정규화
def train_map_func(inp_path, targ_path):
    inp = np.load(inp_path)
    inp = inp.astype(np.float32)/255
    targ = np.load(targ_path)
    targ = targ.astype(np.float32)/255
    inp, targ = augmentation(inp, targ) # train 데이터 증강
    return inp, targ

#augmentation
def augmentation(inp, targ):
    inp, targ = random_rot(inp, targ)
    inp, targ = random_flip_LR(inp, targ)
    inp, targ = random_flip_UD(inp, targ)
    inp, targ = random_crop(inp, targ)
    return inp, targ

def random_rot(inp, targ): #rotate
    k = np.random.randint(4)
    inp = np.rot90(inp, k)
    targ = np.rot90(targ, k)
    return inp, targ

def random_flip_LR(inp, targ):# flip left, right
    f = np.random.randint(2)
    if f == 0:
        inp = np.fliplr(inp)
        targ = np.fliplr(targ) 
    return inp, targ

#추가
def random_flip_UD(inp, targ):# flip up, down
    f = np.random.randint(2)
    if f == 0:
        inp = np.flipud(inp)
        targ = np.flipud(targ) 
    return inp, targ

def random_crop(inp, targ, crop_size=(256, 256)):
    h, w, _ = inp.shape

    top = np.random.randint(0, h - crop_size[0])
    left = np.random.randint(0, w - crop_size[1])
    
    bottom = top + crop_size[0]
    right = left + crop_size[1]

    inp = inp[top:bottom, left:right]
    targ = targ[top:bottom, left:right]
    return inp, targ




In [24]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_inp_files, train_targ_files))
train_dataset = train_dataset.map(lambda item1, item2: tf.numpy_function(train_map_func, [item1, item2], [tf.float32, tf.float32]), num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# batch size, height, width, channle
next(iter(train_dataset))[0].shape, next(iter(train_dataset))[1].shape

InvalidArgumentError: ValueError: high <= 0
Traceback (most recent call last):

  File "c:\Users\AAI\AppData\Local\anaconda3\envs\tensorflow_gpu\lib\site-packages\tensorflow\python\ops\script_ops.py", line 249, in __call__
    ret = func(*args)

  File "c:\Users\AAI\AppData\Local\anaconda3\envs\tensorflow_gpu\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 645, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\AAI\AppData\Local\Temp\ipykernel_13408\3440520456.py", line 7, in train_map_func
    inp, targ = augmentation(inp, targ) # train 데이터 증강

  File "C:\Users\AAI\AppData\Local\Temp\ipykernel_13408\3440520456.py", line 15, in augmentation
    inp, targ = random_crop(inp, targ)

  File "C:\Users\AAI\AppData\Local\Temp\ipykernel_13408\3440520456.py", line 42, in random_crop
    top = np.random.randint(0, h - crop_size[0])

  File "mtrand.pyx", line 746, in numpy.random.mtrand.RandomState.randint

  File "_bounded_integers.pyx", line 1338, in numpy.random._bo

In [ ]:
#validation 정규화
def val_map_func(inp_path, targ_path):
    inp = np.load(inp_path)
    inp = inp.astype(np.float32)/255
    targ = np.load(targ_path)
    targ = targ.astype(np.float32)/255
    return inp, targ

val_dataset = tf.data.Dataset.from_tensor_slices((val_inp_files, val_targ_files))
val_dataset = val_dataset.map(lambda item1, item2: tf.numpy_function(val_map_func, [item1, item2], [tf.float32, tf.float32]), num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# batch size, height, width, channle
next(iter(val_dataset))[0].shape, next(iter(val_dataset))[1].shape

## 2. 모델

1. 모델 load

In [ ]:
model_name='LayerNormalization'

def create_model(model_name):
    from models.LayerNormalization import LayerNormalization
    model = LayerNormalization(input_shape=(img_size, img_size, 3))
    
    # 모델 구조 이미지 저장
    plot_model(model, to_file=model_name+'.png', show_shapes=True)
    return model

model = create_model(model_name) 

2.loss, Optimizer

In [ ]:
def psnr(y_true, y_pred, max_pixel = 1.0):
    return tf.image.psnr(y_true, y_pred, max_val=max_pixel)

loss_name='mae'
# from loss.psnr_loss import psnr_loss
# loss=psnr_loss

optimizer_name='AdamW'
from tensorflow_addons.optimizers import AdamW
optimizer = AdamW(learning_rate)

model.compile(loss='mae', optimizer=optimizer, metrics=psnr)

3. training

In [ ]:
save_path_name=model_name+'_'+loss_name+'_'+optimizer_name

In [ ]:
callbacks_list = [tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join('save_models', save_path_name+'.h5'), 
                                                     monitor='val_loss', save_best_only=True)]

logdir = "logs/"+model_name+'_'+loss_name+'_'+optimizer_name+ datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1, update_freq='epoch')
hist = model.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset, callbacks=[callbacks_list, tensorboard_callback])

In [ ]:
model.save(os.path.join('save_models',save_path_name+'.h5'), custom_objects={'psnr':psnr})

In [ ]:
plt.plot(hist.history["loss"], label='train_loss')
plt.plot(hist.history["val_loss"], label='val_loss')
plt.title('loss_plot')
plt.legend()
plt.show()

## 추론 함수 정의

In [ ]:
model=tf.keras.models.load_model(os.path.join('save_models', save_path_name+'.h5'),  custom_objects={'psnr': psnr})

# batch_size=128
def predict(img_paths, stride=32, batch_size=64):
    results = []
    for img_path in img_paths:
        img = cv2.imread(img_path)
        img = img.astype(np.float32)/255
        crop = []
        position = []
        batch_count = 0

        result_img = np.zeros_like(img)
        voting_mask = np.zeros_like(img)

        for top in tqdm(range(0, img.shape[0], stride)):
            for left in range(0, img.shape[1], stride):
                piece = np.zeros([img_size, img_size, 3], np.float32)
                temp = img[top:top+img_size, left:left+img_size, :]
                piece[:temp.shape[0], :temp.shape[1], :] = temp
                crop.append(piece)
                position.append([top, left])
                batch_count += 1
                if batch_count == batch_size:
                    crop = np.array(crop)
                    pred = model(crop)*255
                    crop = []
                    batch_count = 0
                    for num, (t, l) in enumerate(position):
                        piece = pred[num]
                        h, w, c = result_img[t:t+img_size, l:l+img_size, :].shape
                        result_img[t:t+img_size, l:l+img_size, :] += piece[:h, :w]
                        voting_mask[t:t+img_size, l:l+img_size, :] += 1
                    position = []
        
        result_img = result_img/voting_mask
        result_img = result_img.astype(np.uint8)
        
        
        results.append(result_img)
        
    return results

## ValidationSet 결과

In [ ]:
#전체 이미지에 대한 실험 결과
result = predict(val_input_files[:5], stride=32)

In [ ]:
for i, (input_path, label_path) in enumerate(zip(val_input_files[:5], val_label_files[:5])):
    input_img = cv2.imread(input_path)
    input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
    targ_img = cv2.imread(label_path)
    targ_img = cv2.cvtColor(targ_img, cv2.COLOR_BGR2RGB)
    pred_img = result[i]
    pred_img = cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(20,10))
    plt.subplot(1,3,1)
    plt.imshow(input_img)
    plt.title('input_img', fontsize=10)
    plt.subplot(1,3,2)
    plt.imshow(pred_img)
    plt.title('output_img', fontsize=10)
    plt.subplot(1,3,3)
    plt.imshow(targ_img)
    plt.title('target_img', fontsize=10)
    plt.show()
    print('input PSNR :', psnr(input_img.astype(float), targ_img.astype(float), 255).numpy())
    print('output PSNR :', psnr(result[i].astype(float), targ_img.astype(float), 255).numpy(), '\n')

## TestSet 추론

In [ ]:
test_input_files = './data/test_input_img/'+test_csv['input_img']

In [ ]:
test_result = predict(test_input_files, stride=32)

In [ ]:
for i, input_path in enumerate(test_input_files):
    input_img = cv2.imread(input_path)
    input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
    pred_img = test_result[i]
    pred_img = cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(20,10))
    plt.subplot(1,2,1)
    plt.imshow(input_img)
    plt.title('input_img', fontsize=10)
    plt.subplot(1,2,2)
    plt.imshow(pred_img)
    plt.title('output_img', fontsize=10)
    plt.show()

## 결과 제출

In [ ]:
def make_submission(result):
    os.makedirs('submission/'+save_path_name, exist_ok=True)
    os.chdir("./submission/"+save_path_name+"/")
    sub_imgs = []
    for i, img in enumerate(result):
        path = f'test_{20000+i}.png'
        cv2.imwrite(path, img)
        sub_imgs.append(path)
    submission = zipfile.ZipFile("submission.zip", 'w')
    for path in sub_imgs:
        submission.write(path)
    submission.close()

In [ ]:
make_submission(test_result)